# Лабораторная работа №8. Кластеризация данных. Обучение без учителя

Если обучающий набор представляет собой множество образов 
$$
T = \{x_i\}_{i=1}^N,
$$ 
для которых правильная принадлежность к классам неизвестна, то в этом случае задачу разделения образов $x_i$ на классы (количество которых также может быть известно) называют задачей кластеризации данных, а построение классификатора — обучением без учителя.

Более формально задачу кластеризации можно сформулировать в общем виде следующим образом. Пусть в пространстве векторов признаков $\mathcal{X}$ задана некоторая обучающая выборка 
$$
T = \{x_1, \dots, x_N\} \subset \mathcal{X}
$$ 
и определена метрика (расстояние) $\rho(x_a, x_b)$. Необходимо разделить множество векторов $ T $ на непересекающиеся подмножества $C_k$ (назовём их **кластерами**):
$$
T = \bigcup_{k=1}^K C_k, \quad \bigcap_{k=1}^K C_k = \varnothing,
$$
так, чтобы векторы, принадлежащие одному и тому же кластеру, находились друг к другу на близком расстоянии (в смысле заданной метрики $\rho$. При этом количество кластеров $\mathcal{X}$ может быть как заданным заранее, так и быть найденным в результате решения задачи кластеризации.

В рамках лабораторной работы мы рассмотрим два наиболее распространённых подхода к решению задачи кластеризации:
1. **метод иерархической кластеризации на основе связей**;
2. **метод K-средних**.

Для применения первого подхода на основе выбранной метрики (расстояния, связанной) необходимо предварительно задать **функцию межкластерного расстояния**. Для множеств $A = \{a_i \}$ и $B = \{b_i \}$ находить расстояние между ними $D(A, B)$, определяется одним из трёх следующих способов:

1. **Ближайшей связью (single linkage):**
   $$
   D(A, B) = \min_{a \in A; b \in B} \rho(a, b);
   $$

2. **Средней связью (average linkage):**
   $$
   D(A, B) = \frac{1}{|A| |B|} \sum_{a \in A; b \in B} \rho(a, b);
   $$

3. **Дальней связью (complete linkage):**
   $$
   D(A, B) = \max_{a \in A; b \in B} \rho(a, b).
   $$

Метод иерархической кластеризации множества 
$$
T = \{x_i\}_{i=1}^N
$$
описывается в виде следующего несложного алгоритма:

### Шаг 1.
Задать начальное разбиение 
$$
\Omega = \{C_1, \dots, C_N\}
$$
набора $T = \{x_i\}_{i=1}^N$ на одновекторные кластеры 
$$
C_i = \{x_i\}, \quad i = 1, \dots, N.
$$

### Шаг 2.
Найти два ближайших кластера $C_k$ и $C_l$, таких что
$$
D(C_k, C_l) = \min_{C_k, C_l \in \Omega, \, C_k \neq C_l} D(C_k, C_l).
$$

### Шаг 3.
Заменить в разбиении $\Omega$ два ближайших кластера на один новый кластер $C_k \cup C_l$, уменьшив на единицу значение числа кластеров $|\Omega|$.

### Шаг 4.
Если достигнуто условие остановки, то закончить работу, иначе перейти на шаг 2.

Условием завершения работы алгоритма на шаге 4 может выступать либо достижение желаемого (заданного) количества кластеров $|\Omega|$, либо превышение некоторого порога ближайшего межкластерного расстояния:
$$
D(C_k, C_l) > D_{\max}.
$$

Порог часто выбирают, например, как
$$
D_{\max} = \alpha \max_{x_i, x_j \in T} \rho(x_i, x_j), \alpha \in (0,1)
$$


In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import pdist, squareform
from scipy.optimize import linear_sum_assignment
from scipy.cluster.hierarchy import linkage, fcluster
mpl.rcParams['axes.grid'] = True
plt.rcParams['figure.constrained_layout.use'] = True
import pandas as pd

## Задание 0
Прочтите набор данных из директории _dataset_lab8_.
Номер варианта - `mod(<порядковый номер в группе>, 10) + 1` 

In [ ]:
variant_to_load = 10
input_filename = f"dataset/dataset_variant_{variant_to_load}.csv"
loaded_df = pd.read_csv(input_filename)
data = loaded_df[['X1', 'X2']].values
labels = loaded_df['Label'].values

print(f"Данные загружены из файла {input_filename}:")
print(loaded_df.head())

# Визуализация загруженных данных
plt.scatter(data[:, 0], data[:, 1], c=labels, cmap='viridis', s=15)
plt.title(f"Вариант {variant_to_load}")
plt.xlabel("X1")
plt.ylabel("X2")
plt.show()

## Задание №1
Реализуйте функцию оценки точности кластеризации, используя только мэппинг меток и accuracy_score из библиотеки sklearn.
Вы можете реализовать свою версию этой функции. Например, когда производится полный перебор. 

In [ ]:
def match_and_evaluate_accuracy(true_labels, cluster_labels):
    """
    Оценка качества кластеризации по метрике accuracy и создание маппинга меток.

    :param true_labels: numpy.ndarray, истинные метки.
    :param cluster_labels: numpy.ndarray, метки кластеров.
    :return: tuple (float, dict), где:
        - float: значение accuracy.
        - dict: маппинг меток кластеров в истинные метки.
    """
    n_classes = len(np.unique(true_labels))
    n_clusters = len(np.unique(cluster_labels))

    # Инициализация матрицы соответствий
    cost_matrix = np.zeros((n_classes, n_clusters), dtype=int)

    # TODO: Заполните матрицу cost_matrix
    # Для каждой пары (истинная метка, метка кластера) подсчитайте количество совпадений
    for i in range(n_classes):
        for j in range(n_clusters):
            # Подсчитайте, сколько объектов с истинной меткой i получили метку кластера j
            cost_matrix[i, j] = ...  # ПОДСТАВЬТЕ СВОЙ КОД ЗДЕСЬ

    # TODO: Примените Венгерский алгоритм для оптимального сопоставления
    # Используйте функцию `linear_sum_assignment` из `scipy.optimize`
    row_ind, col_ind = ...  # ПОДСТАВЬТЕ СВОЙ КОД ЗДЕСЬ

    mapping = {col_ind[i]: row_ind[i] for i in range(len(row_ind))}
    mapped_labels = np.array([mapping[label] for label in cluster_labels])
    
    # Вычисление accuracy
    accuracy = accuracy_score(true_labels, mapped_labels)

    return accuracy, mapping

true_labels = np.array([0, 0, 1, 1, 2, 2])
predicted_labels = np.array([2, 2, 0, 0, 1, 1])
accuracy, mapping = match_and_evaluate_accuracy(true_labels, predicted_labels)
assert accuracy == 1.0, 'Неверная реализация'

true_labels = np.array([0, 1, 1, 2, 2, 2])
predicted_labels = np.array([1, 0, 0, 2, 2, 2])
accuracy, mapping = match_and_evaluate_accuracy(true_labels, predicted_labels)
assert accuracy == 1.0, 'Неверная реализация'

## Задание №2

Реализуйте алгоритм _иерархической кластеризации на основе связей_.

In [ ]:
class HierarchicalClustering:
    def __init__(self, data, labels):
        """
        Инициализация объекта для кластеризации.

        :param data: numpy.ndarray, данные для кластеризации
        :param labels: numpy.ndarray, истинные метки для оценки accuracy
        """
        self.data = data
        self.labels = labels
        self.n_samples = len(data)
        self.distance_matrix = self._calculate_distance_matrix()
        self.clusters = [[i] for i in range(self.n_samples)]  # Каждый объект в своем кластере

    def _calculate_distance_matrix(self):
        """
        Вычисление матрицы расстояний между всеми парами точек.

        :return: numpy.ndarray, матрица расстояний
        """
        # Вычисляем попарные евклидовы расстояния
        distance_matrix = squareform(pdist(self.data, metric='euclidean'))
        return distance_matrix

    def _compute_cluster_distance(self, cluster1, cluster2, method):
        """
        Вычисление расстояния между двумя кластерами согласно методу связи.

        :param cluster1: list of int, индексы точек в первом кластере
        :param cluster2: list of int, индексы точек во втором кластере
        :param method: str, тип связи ('single', 'complete', 'average')
        :return: float, расстояние между кластерами
        """
        distances = self.distance_matrix[np.ix_(cluster1, cluster2)]
        if method == 'single':
            return ... # ПОДСТАВЬТЕ СВОЙ КОД ЗДЕСЬ
        elif method == 'complete':
            return ... # ПОДСТАВЬТЕ СВОЙ КОД ЗДЕСЬ
        elif method == 'average':
            return np.mean(distances)
        else:
            raise ValueError(f"Unknown linkage method: {method}")

    def fit(self, method='average'):
        """
        Выполнение кластеризации на основе иерархического алгоритма.

        :param method: str, тип связи ('single', 'complete', 'average')
        :return: numpy.ndarray, метки кластеров
        """
        clusters = self.clusters.copy()
        while len(clusters) > 3:
            min_dist = np.inf
            pair_to_merge = None
            # Найти ближайшую пару кластеров
            for i in range(len(clusters)):
                for j in range(i + 1, len(clusters)):
                    dist = self._compute_cluster_distance(clusters[i], clusters[j], method)
                    if dist < min_dist:
                        min_dist = dist
                        pair_to_merge = (i, j)
            if pair_to_merge is None:
                break  # Нет пар для объединения
            i, j = pair_to_merge
            # Объединить кластеры
            new_cluster = clusters[i] + clusters[j]
            # Удалить старые кластеры и добавить новый
            clusters.pop(j)
            clusters.pop(i)
            clusters.append(new_cluster)
        # Присвоить метки кластерам
        cluster_labels = np.zeros(self.n_samples, dtype=int)
        for idx, cluster in enumerate(clusters):
            for sample_idx in cluster:
                cluster_labels[sample_idx] = idx
        return cluster_labels

    def plot_clusters(self, cluster_labels, method):
        """
        Визуализация кластеров.

        :param cluster_labels: numpy.ndarray, метки кластеров
        :param method: str, тип связи ('single', 'complete', 'average')
        """
        plt.figure(figsize=(8, 6))
        plt.scatter(self.data[:, 0], self.data[:, 1], c=cluster_labels, cmap='viridis', s=15)
        plt.title(f'Кластеры с использованием {method}')
        plt.xlabel("X1")
        plt.ylabel("X2")
        plt.show()

# Генерация данных для тестирования
np.random.seed(42)
test_data, test_labels = make_blobs(n_samples=150, centers=3, cluster_std=1.0, random_state=42)
# Тестирование
results = {}

hc = HierarchicalClustering(test_data, test_labels)
for method in ['single', 'complete', 'average']:
    print(f"Тестирование метода связи: {method}")
    # Кластеризация
    cluster_labels = hc.fit(method=method)
    
    # Оценка качества
    accuracy, _ = match_and_evaluate_accuracy(test_labels, cluster_labels)
    results[method] = accuracy
    
    # Визуализация кластеров
    print(f"Accuracy для {method} linkage: {accuracy:.2f}")

# Проверка корректности работы
for method, acc in results.items():
    assert acc > 0.99, f"Accuracy слишком низкое для {method} linkage: {acc:.2f}"

## Задание №3
Примените метод иерархической кластеризации к вашему набору данных, проанализируйте полученные результаты и сделайте соответствующие выводы.
Оценка качества реализации является ключевым этапом в машинном обучении. Проведите сравнение результатов вашей реализации с ожидаемыми или эталонными значениями, чтобы убедиться в её корректности.

In [ ]:
# Сравнение реализаций
comparison_results = []

hc = HierarchicalClustering(data, labels)

hc.plot_clusters(labels, 'идеальной модели кластеризации')

for method in ['single', 'complete', 'average']:
    # Scipy реализация
    Z = linkage(data, method=method)
    cluster_labels_scipy = fcluster(Z, t=3, criterion='maxclust') - 1
    scipy_accuracy, _ = match_and_evaluate_accuracy(labels, cluster_labels_scipy)
    
    # TODO: Допишите код
    cluster_labels_our = ...  # ПОДСТАВЬТЕ СВОЙ КОД ЗДЕСЬ
    our_accuracy, _ = match_and_evaluate_accuracy(labels, ... ) # ПОДСТАВЬТЕ СВОЙ КОД ЗДЕСЬ
    hc.plot_clusters(cluster_labels_our, method)

    # Сохранение результатов
    comparison_results.append({
        "method": method,
        "scipy_accuracy": scipy_accuracy,
        "our_accuracy": our_accuracy,
    })

# Вывод результатов
for result in comparison_results:
    print(f"Метод: {result['method']}")
    print(f"  - Точность (Scipy): {result['scipy_accuracy']:.4f}")
    print(f"  - Точность (Наша реализация): {result['our_accuracy']:.4f}")

**Вопросы:**

1. Какой метод показал наилучший результат по метрике точности? Почему, на ваш взгляд, этот метод оказался более эффективным?
1. Какие ошибки классификации наблюдаются для каждого метода? С чем они могут быть связаны?
1. Влияет ли исходное распределение данных (например, наличие шумов или перекрытия классов) на качество кластеризации? Если да, то каким образом?
1. Привидите пример, при котором все три вида связей будут давать один и тот же результат кластеризации. Пример должен объяснять суть, а не быть набором значений.

**Ответы:**

1. ...
1. ...

Альтернативой иерархической кластеризации является метод **K-средних**. В его базовом варианте количество кластеров $K$ задаётся заранее как входной параметр, а расстояние между векторами $( \mathbf{x}, \mathbf{y})$ понимается в смысле евклидовой метрики:
$$
\rho(\mathbf{x}, \mathbf{y}) = \|\mathbf{x} - \mathbf{y}\| = \sqrt{(\mathbf{x} - \mathbf{y})^\top (\mathbf{x} - \mathbf{y})}.
$$

Разбиение обучающего набора $T = \{\mathbf{x}_1, \dots, \mathbf{x}_N\} \subset \mathcal{X}$ на кластеры $\{C_1, \dots, C_K\}$ 
ищется из условия минимизации среднего квадрата внутрикластерных расстояний между векторами с функцией штрафа:
$$
E = \sum_{i=1}^K \frac{1}{|C_i|} \sum_{\mathbf{x}_k, \mathbf{x}_m \in C_i} |\mathbf{x}_k - \mathbf{x}_m\|^2 \to \min_{\{C_1, \dots, C_K\}}. \tag{6.2}
$$

Для пояснения алгоритма, реализующего метод  **K-средних**, величины $E_i$ в (6.2) нагляднее представить в другом виде:
$$
E_i = \frac{1}{|C_i|} \sum_{\mathbf{x}_k, \mathbf{x}_m \in C_i} \|\mathbf{x}_k - \mathbf{x}_m\|^2 = \sum_{\mathbf{x}_k \in C_i} \|\mathbf{x}_k - \mathbf{m}_i\|^2, \tag{6.3}
$$
где **центроиды**:
$$
\mathbf{m}_i = \frac{1}{|C_i|} \sum_{\mathbf{x}_k \in C_i} \mathbf{x}_k \tag{6.4}
$$
представляют собой средние арифметические векторов кластеров $C_i$.


Приведём описание алгоритма, реализующего разбиение заданного обучающего набора $T = \{\mathbf{x}_1, \dots, \mathbf{x}_N\}$ 
на кластеры $\{C_1, \dots, C_K\}$ по методу  **K-средних**.

### Шаг 1.
Инициализация: задать начальный набор центроидов 
$$
\{\mathbf{m}_1, \dots, \mathbf{m}_K\} \subset \mathcal{X}.
$$

### Шаг 2.
Сформировать кластеры $\{C_i\}_{i=1}^K$ из ближайших к центроидам векторов обучающей выборки 
$T = \{\mathbf{x}_1, \dots, \mathbf{x}_N\}$ по правилу:
$$
C_i = \{\mathbf{x}_k \in T \mid \|\mathbf{x}_k - \mathbf{m}_i\| = \min_{i=1, \dots, K} \|\mathbf{x}_k - \mathbf{m}_i\|\}. \tag{6.7}
$$

### Шаг 3.
Для кластеров $\{C_i\}_{i=1}^K$ пересчитать их центроиды (6.4):
$$
\mathbf{m}_i = \frac{1}{|C_i|} \sum_{\mathbf{x}_k \in C_i} \mathbf{x}_k.
$$

### Шаг 4.
Если все новые кластеры оказались теми же, что и были ранее, то закончить работу алгоритма, иначе перейти на шаг 2.


## Задание 4
Реализуйте метод кластеризации K-средних. Если вам непонятен шаблон, вы можете реализовать алгоритм удобным для вас способом, но не прибегая к использованию готовых библиотек.

In [ ]:
class KMeansCustom:
    def __init__(self, n_clusters, max_iter=100, tol=1e-4):
        """
        Инициализация K-means.

        :param n_clusters: int, количество кластеров.
        :param max_iter: int, максимальное количество итераций.
        :param tol: float, критерий остановки по изменению центроидов.
        """
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.tol = tol

    def fit(self, X):
        """
        Обучение модели K-means.

        :param X: numpy.ndarray, обучающие данные.
        :return: numpy.ndarray, метки кластеров.
        """
        n_samples, n_features = X.shape
        # TODO: Добавьте случайную инициализацию центроидов
        centroids = ... # ПОДСТАВЬТЕ СВОЙ КОД ЗДЕСЬ

        for iteration in range(self.max_iter):
            # Вычисление расстояний от каждого образца до центроидов
            distances = np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)

            # TODO: Найдите метки для образцов из Х на основе расстояний
            labels = ... # ПОДСТАВЬТЕ СВОЙ КОД ЗДЕСЬ

            # TODO: Обновите (пересчитайте) центроиды
            new_centroids = ... # ПОДСТАВЬТЕ СВОЙ КОД ЗДЕСЬ

            # Проверка критерия остановки
            if np.linalg.norm(new_centroids - centroids) < self.tol:
                break

            centroids = new_centroids

        self.centroids = centroids
        self.labels_ = labels
        return labels

    def predict(self, X):
        """
        Прогнозирование кластеров для новых данных.

        :param X: numpy.ndarray, данные для предсказания.
        :return: numpy.ndarray, метки кластеров.
        """
        distances = np.linalg.norm(X[:, np.newaxis] - self.centroids, axis=2)
        return np.argmin(distances, axis=1)


X = np.array([[1, 1], [1.1, 1.1], [5, 5], [5.1, 5.1], [9, 9], [9.1, 9.1]])
true_labels = np.array([0, 0, 1, 1, 2, 2])  # Истинные метки кластеров

kmeans = KMeansCustom(n_clusters=3, max_iter=100, tol=1e-4)
n_attempts = 10
best_accuracy = 0

for attempt in range(n_attempts):
    kmeans = KMeansCustom(n_clusters=3, max_iter=100, tol=1e-4)
    predicted_labels = kmeans.fit(X)
    accuracy, _ = match_and_evaluate_accuracy(true_labels, predicted_labels)
    best_accuracy = max(best_accuracy, accuracy)


assert best_accuracy == 1.0, f"Тест не пройден: accuracy = {accuracy:.2f}"

## Задание 5.1
Примените метод K-средних к вашему датасету. Поскольку результат работы алгоритма зависит от начального (случайного) разбиения на кластеры, выполните несколько запусков алгоритма. В качестве итогового результата выберите кластеризацию с минимальной суммой евклидовых расстояний от точек до их центроидов (inertia).

In [ ]:
# Многократный запуск K-means
n_runs = 10
best_labels = None
best_inertia = float('inf')
best_centroids = None

for run in range(n_runs):
    kmeans = KMeansCustom(n_clusters=3)
    
    predicted_labels = kmeans.fit(data)
    # TODO: Добавьте вычисление Евклидовой ошибки (сумма расстояний до ближайших центроидов)
    inertia = ... # ПОДСТАВЬТЕ СВОЙ КОД ЗДЕСЬ

    if inertia < best_inertia:
        best_inertia = inertia
        best_labels = predicted_labels
        best_centroids = kmeans.centroids

accuracy, mapping = match_and_evaluate_accuracy(labels, best_labels)
print("Accuracy = ", accuracy)

Визулизируйте результаты кластеризации. Для этого вам необходимо дописать код в функции *visualize_clusters*

In [ ]:
def visualize_clusters(data, true_labels, predicted_labels, mapping, centroids):
    """
    Визуализация кластеров с выделением ошибок.

    :param data: numpy.ndarray, данные.
    :param true_labels: numpy.ndarray, истинные метки.
    :param predicted_labels: numpy.ndarray, предсказанные метки.
    :param mapping: dict, сопоставление меток кластеров истинным меткам.
    :param centroids: numpy.ndarray, координаты центроидов.
    """
    # Преобразуем предсказанные метки в истинные метки на основе сопоставления
    mapped_labels = np.array([mapping[label] for label in predicted_labels])

    # Определяем ошибки
    errors = mapped_labels != true_labels

    plt.figure(figsize=(8, 6))

    for i in np.unique(true_labels):
        # Корректно классифицированные точки
        plt.scatter(
            data[(true_labels == i) & ~errors, 0], 
            data[(true_labels == i) & ~errors, 1], 
            label=f'Класс {i}', alpha=1.0
        )
        # TODO: Добавьте визуализацию для ошибок кластеризации
        # Ошибочные точки того же класса
        plt.scatter(...)  # ПОДСТАВЬТЕ СВОЙ КОД ЗДЕСЬ

    # центроиды
    plt.scatter(
        centroids[:, 0], 
        centroids[:, 1], 
        c='black', 
        marker='X', 
        s=150, 
        label='Центроиды'
    )

    plt.title('Результаты кластеризации')
    plt.xlabel('Признак 1')
    plt.ylabel('Признак 2')
    plt.legend()
    plt.show()

visualize_clusters(data, labels, best_labels, mapping, best_centroids)

**Вопросы:**

1. Почему начальная случайная инициализация центроидов может повлиять на результаты работы метода K-средних? Как можно исправить ситуация, не прибегая к нескольким запускам алгоритма ? 
1. Что произойдёт, если число кластеров K будет больше или меньше, чем фактическое число групп в данных?
1. Как поведёт себя метод K-средних в случае U-образных или вытянутых кластеров ? Для ответа на этот вопрос вам придётся дописать код ниже
1. Сравните точность метода K-средних и метода иерархической кластеризации. Кто оказался точнее и почему? 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, make_blobs

# Генерация данных: вытянутые (U-образные) кластеры
u_shaped_data, u_shaped_labels = make_moons(n_samples=500, noise=0.05, random_state=42)
plt.figure(figsize=(12, 6))

plt.scatter(u_shaped_data[:, 0], u_shaped_data[:, 1], c=u_shaped_labels, cmap='viridis', s=20)
plt.title("U-образные кластеры")
plt.xlabel("X1")
plt.ylabel("X2")

#Ваш код для обучения и визуализции

**Ответы:**

1. ...
1. ...
1. ...

## Задание 5.2
Сравните вашу реализацию с той, что доступна в пакете sklearn. Допишите код, сравните точность и сделайте выводы.

In [ ]:
from sklearn.cluster import KMeans
# Ваш код

**Ваши выводы:**

### Модели распределения в виде гауссовых смесей
До этого при решении задачи кластеризации мы не привлекали каких-либо моделей для описания законов распределения данных. Вместе с тем на практике могут быть известны как априорные сведения о количестве классов $K$, которые требуется выделить (т.е. известно число классов, представленных в выборке), так и обоснованное предположение о виде внутриклассовых распределений. Эту информацию, конечно, следует использовать.
Если предположить, что разделения описываются функциями плотности вероятностей $p(x \mid C_k) $, то для генеральной совокупности, из которой получена обучающая выборка, плотность вероятности будет иметь вид:
$$
p(x) = \sum_{k=1}^K \alpha_k p(x \mid C_k), \tag{6.17}
$$
где$\sum_{k=1}^K \alpha_k = 1.$ Весовой коэффициент $\alpha_k$ в (6.17) определяет долю класса $C_k$ в общем распределении, которую можно также интерпретировать как априорную вероятность появления этого класса: $\alpha_k = P(C_k).$

Положим, что в заданной обучающей выборке $T = \{\mathbf{x}_j\}_{j=1}^N$
представлены образцы из $K$ классов $\{C_1, \dots, C_K\} \$, каждый из которых имеет многомерное нормальное распределение. Тогда плотность распределения для генеральной совокупности принимает вид гауссовой смеси:
$$
p(x) = \sum_{k=1}^K \alpha_k \mathcal{N}(x \mid \mathbf{m}_k, C_k). \tag{6.18}
$$


Далее мы будем исходить из того, что в (6.18) параметры плотностей  $p(x \mid C_k) = \mathcal{N}(x \mid \mathbf{m}_k, C_k)$
(векторы математических ожиданий $\mathbf{m}_k$ и ковариационные матрицы $C_k$, веса классов $\alpha_k$ и принадлежность того или иного образа обучающей выборки $T = \{\mathbf{x}_j\}_{j=1}^N$ к тому или иному классу неизвестны.

Если бы в выборке $T$ содержались метки принадлежности векторов $\mathbf{x}_n$ конкретным классам, то найти по ней оценки параметров
$$
\Theta = \{\alpha_k, \mathbf{m}_k, C_k\}_{k=1}^K
$$
функции (6.18) можно было бы в результате статистического анализа данных. Обозначим множество векторов обучающей выборки, принадлежащих классу $C_k$, как
$$
\Omega_k = \{\mathbf{x}_n \in T \mid y_n = k\},
$$
а количество этих векторов как $N_k = |\Omega_k|$. Для нахождения параметров плотности (6.18) можем использовать следующие хорошо известные из математической статистики точечные оценки \[6\]:
$$
\alpha_k = \frac{N_k}{N}, \quad
\mathbf{m}_k = \frac{1}{N_k} \sum_{\mathbf{x}_n \in \Omega_k} \mathbf{x}_n, \quad
\hat{C}_k = \frac{1}{N_k-1} \sum_{\mathbf{x}_n \in \Omega_k} (\mathbf{x}_n - \mathbf{m}_k)(\mathbf{x}_n - \mathbf{m}_k)^\top, \quad k = 1, \dots, K. \tag{6.19}
$$

Напомним, что при известном (не требующем оценивания) векторе математических ожиданий $\mathbf{m}_k$ вместо (6.19) для ковариационной матрицы используется оценка:
$$
\hat{C}_k = \frac{1}{N_k} \sum_{\mathbf{x}_n \in \Omega_k} (\mathbf{x}_n - \mathbf{m}_k)(\mathbf{x}_n - \mathbf{m}_k)^\top. \tag{6.20}
$$

В рассматриваемой нами задаче метки принадлежности к классам скрыты (неизвестны), и применить напрямую формулы (6.20) мы не можем.


Опустив длинные выкладки, приведём ряд формул, которые нам пригодятся:

$$
\gamma_{n,k} = P(C_k \mid \mathbf{x}_n) = \frac{\alpha_k \mathcal{N}(\mathbf{x}_n \mid \mathbf{m}_k, C_k)}{\sum_{i=1}^K \alpha_i \mathcal{N}(\mathbf{x}_n \mid \mathbf{m}_i, C_i)}, \tag{6.24}
$$
— апостериорная вероятность принадлежности вектора $\mathbf{x}_n$ к классу $C_k$.

$$
\mathbf{m}_k = \frac{1}{\tilde{N}_k} \sum_{n=1}^N \gamma_{n,k} \mathbf{x}_n, \tag{6.25}
$$
где величину
$$
\tilde{N}_k = \sum_{n=1}^N \gamma_{n,k} = \sum_{n=1}^N P(C_k \mid \mathbf{x}_n) \tag{6.26}
$$

Обратим внимание на сходство и различия формулы (6.25), которую можно использовать для формирования оценки $\hat{\mathbf{m}}_k$, и выражения для $\mathbf{m}_k$ в (6.19). Поскольку принадлежность векторов выборки к классам при использовании формулы (6.25) неизвестна, то для формирования оценки $\hat{\mathbf{m}}_k$ учитываются уже все векторы из обучающей выборки $T$, но с весами $\gamma_{n,k} = P(C_k \mid \mathbf{x}_n)$, которые характеризуют "уверенность" в принадлежности каждого конкретного вектора $\mathbf{x}_n$ к классу $C_k$.

$$
\hat{C}_k = \frac{1}{\tilde{N}_k} \sum_{n=1}^N \gamma_{n,k} (\mathbf{x}_n - \mathbf{m}_k)(\mathbf{x}_n - \mathbf{m}_k)^\top, \tag{6.27}
$$
где 
$
\tilde{N}_k = \sum_{n=1}^N \gamma_{n,k}.
$
$$
\alpha_k = \frac{1}{N} \sum_{n=1}^N \gamma_{n,k} = \frac{\tilde{N}_k}{N}. \tag{6.28}
$$

Как видим, для нахождения параметров $\{\alpha_k, \mathbf{m}_k, C_k\}_{k=1}^K$ по формулам (6.25)–(6.28) требуется сначала вычислить "уверенности" $\gamma_{n,k}$ по правилу (6.24). Но для использования соотношения (6.24) параметры $\{\alpha_k, \mathbf{m}_k, C_k\}_{k=1}^K$ уже должны быть известны. Этот замкнутый круг естественным образом наводит на мысль об использовании следующей итерационной процедуры: задав каким-либо образом начальные значения параметров $\{\alpha_k^{(0)}, \mathbf{m}_k^{(0)}, C_k^{(0)}\}_{k=1}^K$, рассчитать значения "уверенности" $\gamma_{n,k}$ по (6.24) и после этого найти новые значения параметров
$$
\{\alpha_k^{(1)}, \mathbf{m}_k^{(1)}, C_k^{(1)}\}_{k=1}^K.
$$
Когда найденные на очередной итерации параметры
$$
\{\alpha_k^{(i)}, \mathbf{m}_k^{(i)}, C_k^{(i)}\}_{k=1}^K
$$
становятся мало отличимыми от предыдущих значений
$$
\{\alpha_k^{(i-1)}, \mathbf{m}_k^{(i-1)}, C_k^{(i-1)}\}_{k=1}^K,
$$
итерационный процесс завершается. Альтернативным условием завершения работы алгоритма может служить также малое (меньше некоторого порога) изменение функции логарифмического правдоподобия 
$$
L(\Theta \mid X) = \ln \prod_{n=1}^N p(x_n) = \sum_{n=1}^N \ln \left( \sum_{k=1}^K \alpha_k \mathcal{N}(x_n \mid \mathbf{m}_k, C_k) \right). \tag{6.21}
$$ по сравнению со значением, полученным на предыдущей итерации.

### Шаг 1.
Инициализация: задать начальные значения параметров
$$
\{\alpha_k^{(0)}, \mathbf{m}_k^{(0)}, C_k^{(0)}\}_{k=1}^K,
$$
и рассчитать соответствующее им значение $L_{\text{old}}$ функции логарифмического правдоподобия (6.21).



### Шаг 2. **Expectation step**.
По текущим значениям $\{\alpha_k, \mathbf{m}_k, C_k\}_{k=1}^K$ вычислить по (6.24) условные вероятности — математические ожидания скрытых переменных:
$$
\gamma_{n,k} = P(C_k \mid \mathbf{x}_n) = \mathbb{M}(Z_{n,k}),
$$
для всех индексов $n = 1, \dots, N; \, k = 1, \dots, K $.

### Шаг 3. **Maximization step**.
По текущим значениям $\gamma_{n,k}$ пересчитать по формулам (6.16) — (6.28) значения параметров $\{\alpha_k, \mathbf{m}_k, C_k\}_{k=1}^K$ и вычислить соответствующее им значение $L_{\text{new}}$ функции (6.21).

### Шаг 4. Проверка условия окончания работы.
Если $|L_{\text{new}} - L_{\text{old}}| \leq \varepsilon$, то завершить работу алгоритма; иначе положить $L_{\text{old}} = L_{\text{new}}$ и перейти на шаг 2.

---

Таким образом, шаг 2 — вычисление условных вероятностей $\gamma_{n,k} = P(C_k \mid \mathbf{x}_n)$ — это нахождение математических ожиданий скрытых переменных. По этим значениям на шаге 3 находятся параметры распределения
$$
\Theta = \{\alpha_k, \mathbf{m}_k, C_k\}_{k=1}^K,
$$
которые максимизируют функцию логарифмического правдоподобия (6.21).

## Задание 6.1

Реализуйте EM алгоритм. Примените его для кластеризации ваших данных. Визуализируйте результат. Если вам непонятен шаблон, вы можете реализовать алгоритм удобным для вас способом, но не прибегая к использованию готовых библиотек.

In [ ]:
from scipy.stats import multivariate_normal

class GaussianMixtureEM:
    def __init__(self, n_clusters, max_iter=100, tol=1e-4):
        """
        Инициализация алгоритма EM для гауссовой смеси.

        :param n_clusters: int, количество кластеров.
        :param max_iter: int, максимальное количество итераций.
        :param tol: float, критерий остановки по изменению параметров.
        """
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.tol = tol
        self.weights = None  
        self.means = None  
        self.covariances = None 

    def initialize_parameters(self, X, initial_labels):
        """
        Инициализация параметров смеси на основе начальных меток.

        :param X: numpy.ndarray, данные.
        :param initial_labels: numpy.ndarray, начальные метки кластеров.
        """
        n_samples, n_features = X.shape
        self.weights = np.array([np.mean(initial_labels == k) for k in range(self.n_clusters)])
        self.means = np.array([X[initial_labels == k].mean(axis=0) for k in range(self.n_clusters)])
        self.covariances = np.array([
            np.cov(X[initial_labels == k].T) + np.eye(n_features) * 1e-6
            for k in range(self.n_clusters)
        ])

    def e_step(self, X):
        """
        E-шаг: вычисление responsibilities (апостериорных вероятностей).
        :param X: numpy.ndarray, данные.
        :return: numpy.ndarray, матрица responsibilities.
        """
        n_samples = X.shape[0]
        responsibilities = np.zeros((n_samples, self.n_clusters))
        
        # TODO: Заполнить responsibilities для каждого кластера k
        for k in range(self.n_clusters):
            # Используйте multivariate_normal.pdf для вычисления вероятности принадлежности к кластеру
            responsibilities[:, k] = ...  # ПОДСТАВЬТЕ СВОЙ КОД ЗДЕСЬ
        
        # Нормализовать вероятности, чтобы сумма по строке была равна 1
        responsibilities /= responsibilities.sum(axis=1, keepdims=True)
        return responsibilities

    def m_step(self, X, responsibilities):
        """
        M-шаг: обновление параметров смеси (весов, средних, ковариаций).

        :param X: numpy.ndarray, данные.
        :param responsibilities: numpy.ndarray, матрица responsibilities.
        """
        n_samples, n_features = X.shape
        N_k = responsibilities.sum(axis=0) 

        # TODO: Обновите веса кластеров
        self.weights = ...  # ПОДСТАВЬТЕ СВОЙ КОД ЗДЕСЬ

        # TODO: Обновите средние значения кластеров
        self.means = ...  # ПОДСТАВЬТЕ СВОЙ КОД ЗДЕСЬ

        # TODO: Обновите ковариационные матрицы
        self.covariances = []
        for k in range(self.n_clusters):
            diff = X - self.means[k]
            weighted_diff = responsibilities[:, k][:, np.newaxis] * diff
            covariance = ...  # ПОДСТАВЬТЕ СВОЙ КОД ЗДЕСЬ
            covariance += np.eye(n_features) * 1e-6  # Регуляризация
            self.covariances.append(covariance)
        self.covariances = np.array(self.covariances)

    def compute_log_likelihood(self, X, responsibilities):
        """
        Вычисление логарифмической функции правдоподобия.

        :param X: numpy.ndarray, данные.
        :param responsibilities: numpy.ndarray, матрица responsibilities.
        :return: float, значение логарифмической правдоподобности.
        """
        # TODO: Реализуйте вычисление логарифмической функции правдоподобия
        log_likelihood = ...  # ПОДСТАВЬТЕ СВОЙ КОД ЗДЕСЬ
        return log_likelihood

    def fit(self, X, initial_labels):
        """
        Обучение гауссовой смеси методом EM.

        :param X: numpy.ndarray, данные.
        :param initial_labels: numpy.ndarray, начальные метки кластеров.
        """
        self.initialize_parameters(X, initial_labels)
        log_likelihood = 0

        for iteration in range(self.max_iter):
            # E-шаг
            responsibilities = self.e_step(X)

            # M-шаг
            self.m_step(X, responsibilities)

            # Вычисление логарифмической правдоподобности
            new_log_likelihood = self.compute_log_likelihood(X, responsibilities)
            
            # Критерий остановки
            if abs(new_log_likelihood - log_likelihood) < self.tol:
                print(f"Сходимость достигнута на итерации {iteration}")
                break
            log_likelihood = new_log_likelihood

    def predict(self, X):
        """
        Прогнозирование меток кластеров на основе обученной гауссовой смеси.

        :param X: numpy.ndarray, данные.
        :return: numpy.ndarray, метки кластеров.
        """
        responsibilities = self.e_step(X)
        return np.argmax(responsibilities, axis=1)  # Метка с максимальной вероятностью


# Используем результат K-средних как начальное разбиение
em_model = GaussianMixtureEM(n_clusters=3, tol=1e-20, max_iter=100)
em_model.fit(data, best_labels)

# Байесовская классификация
em_labels = em_model.predict(data)

# Оценка точности
accuracy_em, mapping_em = match_and_evaluate_accuracy(labels, em_labels)
print(f"Accuracy для алгоритма EM: {accuracy_em:.2f}")

# Визуализация результатов
plt.scatter(data[:, 0], data[:, 1], c=em_labels, cmap='viridis', s=20)
plt.title("Результаты EM-кластеризации")
plt.xlabel("X1")
plt.ylabel("X2")
plt.show()

## Задание 6.2 
Этот метод, аналогично KMeans, реализован в библиотеке sklearn. Оцените качество вашей реализации, сравнив её точность с точностью реализации из sklearn.

In [ ]:
from sklearn.mixture import GaussianMixture 
gmm = GaussianMixture(n_components = 3)
gmm_l = GaussianMixture(n_components = 3).fit_predict(data)
match_and_evaluate_accuracy(labels, gmm_l)

**Ваши выводы**:


**Вопросы:**

1. Почему для начальной инициализации параметров используются результаты K-средних? Какие проблемы могут возникнуть при случайной инициализации?
1. В чём отличие EM-алгоритма от K-средних в плане подхода к кластеризации? Какие преимущества предоставляет EM-алгоритм? Что у них общего?
1. Какие предположения делает гауссова смесь о данных? В каких случаях использование EM-алгоритма будет неэффективным?
1. Как поведёт себя алгоритм EM в случае U-образных кластеров ? Для ответа на этот вопрос вам придётся дописать код ниже
1. Сравните точность метода K-средних и EM-алгоритма. Кто оказался точнее? Почему ? 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, make_blobs

# Генерация данных: вытянутые (U-образные) кластеры
u_shaped_data, u_shaped_labels = make_moons(n_samples=500, noise=0.05, random_state=42)
plt.figure(figsize=(12, 6))

plt.scatter(u_shaped_data[:, 0], u_shaped_data[:, 1], c=u_shaped_labels, cmap='viridis', s=20)
plt.title("U-образные кластеры")
plt.xlabel("X1")
plt.ylabel("X2")

#Ваш код для обучения и визуализции

**Ответы:**

1. ...
1. ...
1. ...
1. ...